## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_3 = pd.read_csv("../../바이너리_최종데이터_1004/gcc6/o3/o3_bincore6.csv", index_col=0)
print(bin6_3.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_3.reset_index(inplace=True, drop=True)

print('shape -', bin6_3.shape)
print('reset_index 완료')
print('input data shape')
bin6_3.head()

(33250757, 2)
shape - (33250757, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_3['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_3['label'].value_counts())

256
0    33215560
1       35197
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_3[bin6_3['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_3), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_3', len(ls_idx_bin))

# loc 로 수정필요
bin6_3_Ngram = bin6_3.loc[ls_idx_bin,:].copy()

1126304
나머지 0
최종 길이 1126304
bin6_3 1126304


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_3_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_3.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_3)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_3[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

35197.0
0 28425135
1000 8553196
2000 20640071
3000 5777696
4000 10780394
5000 18690558
6000 17586049
7000 3810022
8000 28216086
9000 23015213
10000 23023458
11000 30058174
12000 20535374
13000 26921300
14000 23355923
15000 29859338
16000 21248947
17000 29098427
18000 2324954
19000 21867426
20000 31726672
21000 4333069
22000 22862434
23000 12174471
24000 32222947
25000 11268781
26000 17001565
27000 31811266
28000 20726027
29000 7838693
30000 11697238
31000 4518801
32000 33077800
33000 30940067
34000 32171097
35000 28187919
완료
35197


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_3['label'].value_counts()[1])

35197
35197


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_3_Ngram])
final.shape

(2252608, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_3_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_3_onehot_Ngram = pd.concat([final['label'], bc6_3_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_3_onehot_Ngram.shape)

원핫인코딩완료
(2252608, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc6_3 = bc6_3_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_3 = bc6_3_onehot_Ngram['label'].to_numpy()
print(x_bc6_3.shape, x_bc6_3.shape)

x_bc6_3 = x_bc6_3.reshape(-1, right_idx, x_bc6_3.shape[1])
y_bc6_3 = y_bc6_3.reshape(-1, right_idx, 1)

print(x_bc6_3.shape, y_bc6_3.shape)

(2252608, 256) (2252608, 256)
(70394, 32, 256) (70394, 32, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_3.shape[0])

x_bc6_3 = x_bc6_3[p]
y_bc6_3 = y_bc6_3[p]

print(x_bc6_3.shape, y_bc6_3.shape)

(70394, 32, 256) (70394, 32, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc6_3, y_bc6_3):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc6_3[train],
               y_bc6_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc6_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc6_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63354 samples
Epoch 1/10
63354/63354 [==============================] - 112s 2ms/sample - loss: 0.0118 - accuracy: 0.9968
Epoch 2/10
63354/63354 [==============================] - 103s 2ms/sample - loss: 0.0010 - accuracy: 0.9997
Epoch 3/10
63354/63354 [==============================] - 115s 2ms/sample - loss: 7.4092e-04 - accuracy: 0.9998
Epoch 4

accuracy_score 0.9999112215909091
recall_score 0.9982032854209446
precision_score 0.9966683751922091
f1_score 0.9974352398050782
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63354 samples
Epoch 1/10
63354/63354 [==============================] - 120s 2ms/sample - loss: 2.4932e-04 - accuracy: 0.9999
Epoch 2/10
63354/63354 [==============================] - 96s 2ms/sample - loss: 2.1825e-04 

accuracy_score 0.9999467329545455
recall_score 0.9974528782475802
precision_score 0.9994895354772844
f1_score 0.998470168281489
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63354 samples
Epoch 1/10
63354/63354 [==============================] - 112s 2ms/sample - loss: 6.9946e-05 - accuracy: 1.0000
Epoch 2/10
63354/63354 [==============================] - 91s 1ms/sample - loss: 6.8295e-05 -

accuracy_score 0.9999644886363637
recall_score 0.998960498960499
precision_score 0.998960498960499
f1_score 0.998960498960499
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63354 samples
Epoch 1/10
63354/63354 [==============================] - 102s 2ms/sample - loss: 2.3023e-05 - accuracy: 1.0000
Epoch 2/10
63354/63354 [==============================] - 89s 1ms/sample - loss: 9.9468e-06 - a

accuracy_score 0.9999911221590909
recall_score 0.9997455470737914
precision_score 0.9997455470737914
f1_score 0.9997455470737914
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 94s 1ms/sample - loss: 1.4225e-05 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 93s 1ms/sample - loss: 1.1675e-05 -

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 107s 2ms/sample - loss: 2.7974e-10 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 83s 1ms/sample - loss: 2.4220e-10 - accuracy: 1.0000
Epoch 3/10
63355/63355 [=================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 101s 2ms/sample - loss: 1.7484e-10 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 88s 1ms/sample - loss: 1.6646e-10 - accuracy: 1.0000
Epoch 3/10
63355/63355 [=================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 135s 2ms/sample - loss: 1.2251e-10 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 112s 2ms/sample - loss: 1.1701e-10 - accuracy: 1.0000
Epoch 3/10
63355/63355 [================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 63355 samples
Epoch 1/10
63355/63355 [==============================] - 123s 2ms/sample - loss: 1.0749e-10 - accuracy: 1.0000
Epoch 2/10
63355/63355 [==============================] - 110s 2ms/sample - loss: 1.0172e-10 - accuracy: 1.0000
Epoch 3/10
63355/63355 [================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9999112215909091, 0.9999467329545455, 0.9999644886363637, 0.9999911221590909, 0.9999955604489275, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [0.9982032854209446, 0.9974528782475802, 0.998960498960499, 0.9997455470737914, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9966683751922091, 0.9994895354772844, 0.998960498960499, 0.9997455470737914, 0.9997440491425647, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9974352398050782, 0.998470168281489, 0.998960498960499, 0.9997455470737914, 0.9998720081914757, 1.0, 1.0, 1.0, 1.0, 1.0]


## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999809125789836
10-Fold Cross_validation. Recall : 0.9994362209702816
10-Fold Cross_validation. Precision : 0.9994608005846348
10-Fold Cross_validation. F1-Score : 0.9994483462312334


In [13]:
model1.save('gcc6_bin_core_s32_h48_o3.h5')
print('save 완료')

save 완료
